In [4]:
from astropy.io import fits
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
import pandas as pd

In [5]:
haloid = 166
instrument = "athena_wfi"
hdul_bkg = fits.open(f"halo_{haloid}_evt_{instrument}_0.7_7.0.fits")
data_bkg = hdul_bkg['EVENTS'].data
header = hdul_bkg['EVENTS'].header

x = data_bkg['X']
y = data_bkg['Y']
energy = data_bkg["Energy"]*1e-3

print(min(energy), max(energy))

0.020015366 14.999991


In [6]:
# Center from header
x0 = header['TCRPX2']  # 4096.5
y0 = header['TCRPX3']  # 4096.5

# Pixel scale (in arcmin/pixel)
pixscale_deg = abs(header['TCDLT2'])  # or TCDLT3, they're equal in mag
pixscale_arcmin = pixscale_deg * 60.0  # deg → arcmin

# Radial distance in pixels → arcmin
radii = np.sqrt((x - x0)**2 + (y - y0)**2) * pixscale_arcmin

In [7]:
# Annular bin edges
edges = np.array([0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1., 1.3333337, 2.0000003, 2.6666663, 4.3333333, 6.0000003, 7.6666663])
trarcmin = (edges[:-1] + edges[1:]) / 2
trwidth = (edges[1:] - edges[:-1]) / 2

In [8]:
# Energy bands in keV
energy_bands = [
    (0.7, 1.0),
    (1.0, 1.3),
    (1.3, 1.6),
    (1.6, 2.0),
    (2.0, 2.7),
    (2.7, 3.4),
    (3.4, 3.8),
    (3.8, 4.3),
    (4.3, 5.0),
    (5.0, 7.0),
]

# Convert to eV if needed (assuming energy is in eV)
for idx, (emin, emax) in enumerate(energy_bands, start=1):

    # Filter photons by energy
    in_band = (energy >= emin) & (energy < emax)

    band_radii = radii[in_band]

    tcounts = []
    tarea = []
    ttexp = []

    for i in range(len(edges) - 1):
        in_annulus = (band_radii >= edges[i]) & (band_radii < edges[i+1])
        count = np.sum(in_annulus)
        area = np.pi * (edges[i+1]**2 - edges[i]**2)
        exposure = 1e5  # Replace with actual exposure if spatially varying
        tcounts.append(count)
        tarea.append(area)
        ttexp.append(exposure)

    df = pd.DataFrame({
        'trarcmin': trarcmin,
        'trwidth': trwidth,
        'tcounts': tcounts,
        'tarea': tarea,
        'ttexp': ttexp
    })

    header_lines = "# trarcmin    trwidth    tcounts      tarea        ttexp\n#\n"
    outname = f"fgbg/fg_profnew_{int(emin*1000):04d}_{int(emax*1000):04d}.dat"


    with open(outname, "w") as f:
        f.write(header_lines)
        df.to_string(f, index=False, header=False, float_format="%.6e")

print("Done generating files for all bands.")

Done generating files for all bands.
